In [2]:
# !wget https://huggingface.co/datasets/mesolitica/chatgpt-malaysian-qa-choice/resolve/main/qa-dewanbahasa-jdbp.jsonl

In [4]:
import json
import random
import re

def clean(string):
    string = re.sub('[^A-Za-z ]+', ' ', string.lower())
    string = re.sub(r'[ ]+', ' ', string).strip()
    return string

def overlap(string1, string2):
    l = set([w for w in clean(string1).split() if len(w) > 2])
    r = set([w for w in clean(string2).split() if len(w) > 2])
    return len(l & r) / len(l)

In [5]:
data = []
with open('qa-dewanbahasa-jdbp.jsonl') as fopen:
    for l in fopen:
        data.append(json.loads(l))

In [6]:
len(data)

2820

In [7]:
indices = {i for i in range(len(data))}

In [27]:
from tqdm import tqdm

pairs = []
for i in tqdm(range(len(data))):
    query = data[i]['paragraph']
    
    for s in data[i]['qa']['qa']:
        pos = [query]
        sampled = random.sample(list(indices - {i}), 100)
        neg = []
        for k in sampled:
            try:
                overlapped = overlap(query, data[k]['paragraph'])
                if overlapped < 0.3:
                    neg.append(data[k]['paragraph'])
                if len(neg) >= 5:
                    break
            except:
                pass
            
        pairs.append({
            'query': s['question'],
            'pos': pos,
            'neg': neg
        })

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2820/2820 [00:37<00:00, 75.40it/s]


In [28]:
len(pairs)

12244

In [31]:
with open('qa-dewanbahasa-jdbp-pair.jsonl', 'w') as fopen:
    for l in pairs:
        if not len(l['pos']) or not len(l['neg']):
            continue
        fopen.write(f'{json.dumps(l)}\n')

In [32]:
from huggingface_hub import HfApi
api = HfApi()
api.upload_file(
    path_or_fileobj='qa-dewanbahasa-jdbp-pair.jsonl',
    path_in_repo='qa-dewanbahasa-jdbp-pair.jsonl',
    repo_id='mesolitica/instructions-pair-mining',
    repo_type='dataset',
)

qa-dewanbahasa-jdbp-pair.jsonl:   0%|          | 0.00/217M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/mesolitica/instructions-pair-mining/commit/de396682a390ad5e89d3e58382ef42025fe127e9', commit_message='Upload qa-dewanbahasa-jdbp-pair.jsonl with huggingface_hub', commit_description='', oid='de396682a390ad5e89d3e58382ef42025fe127e9', pr_url=None, pr_revision=None, pr_num=None)